<a href="https://colab.research.google.com/github/ConsueloFarias/proyecto-final-consuelo-v2/blob/main/clasesCoder/clase-4/Pandas_advance_group_and_agg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dominando Pandas - Agrupación y Filtrado Avanzado

1. Configuración Inicial: Cargar Pandas y el dataset.
2. Exploración Básica: Entender la estructura de nuestros datos.
3. Filtrado con Índices Booleanos: Seleccionar datos basados en condiciones.
  * Condiciones simples.
  * Condiciones múltiples (&, |).
  * Uso de .isin().
4. Agrupación con groupby(): El concepto de "Split-Apply-Combine".
  * Agrupación simple y agregación básica (mean, sum, count, etc.).
  * Agrupación por múltiples columnas.
  * Agregaciones múltiples con .agg().
5. Combinando Filtrado y Agrupación: Potencia al máximo.
6. Conclusiones y Próximos Pasos.

## 1. Configuracion inicial

In [ ]:
import pandas as pd

# URL del archivo CSV
url = 'https://raw.githubusercontent.com/JJTorresDS/stocks-ds-edu/main/stocks.csv'

df_stocks = pd.read_csv(url)

In [ ]:
# Mostrar las primeras 5 filas para ver cómo lucen los datos
df_stocks.head()

,formatted_date,MCD,SBUX,GOOG,AMZN,MSFT,JPM,BAC,C,MAR,HLT,RCL,V,MA,PYPL
0,2016-01-01,106.332146,54.353962,742.950012,587.000000,49.853489,50.424938,12.573010,36.897804,57.754189,35.192841,74.235298,71.574371,85.822624,36.139999
1,2016-02-01,100.671043,52.064243,697.770020,552.520020,46.043667,48.033066,11.132540,33.707108,64.228912,41.061607,67.360649,69.556580,83.956566,38.139999
2,2016-03-01,108.782211,53.571442,744.950012,593.640015,50.339031,50.524323,12.021718,36.223217,67.336624,44.499886,74.790009,73.631477,91.278160,38.599998
3,2016-04-01,109.483307,50.457645,693.010010,659.590027,45.453705,53.919910,12.995729,40.153545,66.305466,43.716049,70.465584,74.363144,93.683258,39.180000
4,2016-05-01,105.648926,49.255203,735.719971,722.789978,48.306515,56.098225,13.201019,40.447887,62.474155,41.198154,70.456474,75.999847,92.817329,37.790001


In [ ]:
# Mostrar información general del DataFrame (tipos de datos, nulos, etc.)
df_stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   formatted_date  71 non-null     object 
 1   MCD             71 non-null     float64
 2   SBUX            71 non-null     float64
 3   GOOG            71 non-null     float64
 4   AMZN            71 non-null     float64
 5   MSFT            71 non-null     float64
 6   JPM             71 non-null     float64
 7   BAC             71 non-null     float64
 8   C               71 non-null     float64
 9   MAR             71 non-null     float64
 10  HLT             71 non-null     float64
 11  RCL             71 non-null     float64
 12  V               71 non-null     float64
 13  MA              71 non-null     float64
 14  PYPL            71 non-null     float64
dtypes: float64(14), object(1)
memory usage: 8.4+ KB


In [ ]:
# Es buena práctica convertir la columna 'Date' a tipo datetime
# Si 'Date' no es tipo datetime, muchas operaciones de tiempo fallarán o serán ineficientes.
df_stocks['formatted_date'] = pd.to_datetime(df_stocks['formatted_date'])

In [ ]:
df_stocks.dtypes['formatted_date']

dtype('<M8[ns]')

## 2. Exploracion basica

In [ ]:
# Ver las dimensiones del DataFrame (filas, columnas)
print(f"\nDimensiones del DataFrame: {df_stocks.shape}")


Dimensiones del DataFrame: (71, 15)


In [ ]:
# Ver las columnas disponibles
print(f"\nColumnas: {df_stocks.columns.tolist()}")


Columnas: ['formatted_date', 'MCD', 'SBUX', 'GOOG', 'AMZN', 'MSFT', 'JPM', 'BAC', 'C', 'MAR', 'HLT', 'RCL', 'V', 'MA', 'PYPL']


In [ ]:
# Obtener estadísticas descriptivas básicas para las columnas numéricas
print("\nEstadísticas Descriptivas:")
df_stocks[['MCD', 'SBUX', 'GOOG', 'AMZN', 'MSFT', 'JPM', 'BAC', 'C', 'MAR', 'HLT', 'RCL', 'V', 'MA', 'PYPL']].describe()


Estadísticas Descriptivas:


,MCD,SBUX,GOOG,AMZN,MSFT,JPM,BAC,C,MAR,HLT,RCL,V,MA,PYPL
count,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000
mean,166.141364,70.467695,1309.236057,1844.628025,133.132113,99.241065,25.946568,56.735507,111.312456,81.478392,90.851429,145.278413,218.651822,115.617464
std,42.876054,22.053342,583.436550,944.397337,77.765768,30.235695,8.089168,10.997466,28.170181,26.520324,22.909133,52.414162,95.469659,75.914087
min,98.913841,45.652046,692.099976,552.520020,45.453705,48.033066,11.132540,33.707108,57.754189,35.192841,31.853811,69.556580,83.956566,36.139999
25%,137.941452,52.327950,934.915009,974.299988,66.861931,80.640980,21.816364,48.657539,89.519512,62.162132,72.517128,94.559734,122.293816,56.109999
50%,165.378510,62.786278,1119.920044,1735.910034,107.740570,96.035423,25.971552,59.150528,122.888184,78.211067,93.269997,143.700562,211.931427,88.760002
75%,202.356850,84.434284,1431.575012,2616.410034,190.287773,109.950825,29.027129,64.777954,133.650467,96.858334,111.008904,190.040054,297.106628,164.619995
max,244.196198,120.441193,2965.409912,3507.070068,331.014404,168.872391,47.551128,77.564484,160.020004,143.949997,131.389603,245.622177,384.999420,291.480011


## 3. Filtrado con indices Booleanos

El filtrado booleano sigue usando condiciones para generar True/False, pero las aplicaremos a la columna de fecha o a las columnas específicas de los tickers.

### 3.1 Filtrar por Fecha

In [ ]:
condicion_fecha = df_stocks['formatted_date'] >= pd.Timestamp('2016-03-01')
df_desde_marzo = df_stocks[condicion_fecha]

In [ ]:
df_desde_marzo.head()

,formatted_date,MCD,SBUX,GOOG,AMZN,MSFT,JPM,BAC,C,MAR,HLT,RCL,V,MA,PYPL
2,2016-03-01,108.782211,53.571442,744.950012,593.640015,50.339031,50.524323,12.021718,36.223217,67.336624,44.499886,74.790009,73.631477,91.278160,38.599998
3,2016-04-01,109.483307,50.457645,693.010010,659.590027,45.453705,53.919910,12.995729,40.153545,66.305466,43.716049,70.465584,74.363144,93.683258,39.180000
4,2016-05-01,105.648926,49.255203,735.719971,722.789978,48.306515,56.098225,13.201019,40.447887,62.474155,41.198154,70.456474,75.999847,92.817329,37.790001
5,2016-06-01,104.160179,51.435623,692.099976,715.619995,46.964779,53.408058,11.844324,36.817394,63.153168,44.811901,61.431141,71.533958,85.229347,36.509998
6,2016-07-01,102.580368,52.273071,768.789978,758.809998,52.021957,54.980907,12.977122,38.050716,68.132416,46.124634,66.270622,75.276054,92.178528,37.240002


### 3.2 Filtrar por el Valor de un Ticker Específico

In [ ]:
condicion_goog_alto = df_stocks['GOOG'] > 700
df_goog_alto = df_stocks[condicion_goog_alto]

In [ ]:
df_goog_alto[["formatted_date","GOOG"]].head()

,formatted_date,GOOG
0,2016-01-01,742.950012
2,2016-03-01,744.950012
4,2016-05-01,735.719971
6,2016-07-01,768.789978
7,2016-08-01,767.049988


In [ ]:
condicion_msft_bajo = df_stocks['MSFT'] < 48
df_msft_bajo = df_stocks[condicion_msft_bajo]

In [ ]:
df_msft_bajo[["formatted_date", "MSFT"]].head()

,formatted_date,MSFT
1,2016-02-01,46.043667
3,2016-04-01,45.453705
5,2016-06-01,46.964779


### 3.3 Condiciones Múltiples (&, |):

In [ ]:
df_combo1 = df_stocks[(df_stocks['GOOG'] > 700) & (df_stocks['MSFT'] < 50)]

In [ ]:
df_combo1[["formatted_date", "GOOG", "MSFT"]].head()

,formatted_date,GOOG,MSFT
0,2016-01-01,742.950012,49.853489
4,2016-05-01,735.719971,48.306515


In [ ]:
condicion_fecha_marzo = df_stocks['formatted_date'] >= '2016-03-01'
condicion_amzn_alto = df_stocks['AMZN'] > 600
df_combo2 = df_stocks[condicion_fecha_marzo | condicion_amzn_alto]

In [ ]:
df_combo2[["formatted_date", "AMZN"]]

,formatted_date,AMZN
2,2016-03-01,593.640015
3,2016-04-01,659.590027
4,2016-05-01,722.789978
5,2016-06-01,715.619995
6,2016-07-01,758.809998
...,...,...
66,2021-07-01,3327.590088
67,2021-08-01,3470.790039
68,2021-09-01,3285.040039
69,2021-10-01,3372.429932


### Selección de Columnas Específicas (Tickers):

In [ ]:
columnas_seleccionadas = ['formatted_date', 'GOOG', 'MSFT']
df_seleccion = df_stocks[columnas_seleccionadas]

df_seleccion.head()

,formatted_date,GOOG,MSFT
0,2016-01-01,742.950012,49.853489
1,2016-02-01,697.770020,46.043667
2,2016-03-01,744.950012,50.339031
3,2016-04-01,693.010010,45.453705
4,2016-05-01,735.719971,48.306515


In [ ]:
df_filtrado_seleccionado = df_stocks[df_stocks['GOOG'] > 700][['formatted_date', 'GOOG', 'MSFT']]

df_filtrado_seleccionado.head()

,formatted_date,GOOG,MSFT
0,2016-01-01,742.950012,49.853489
2,2016-03-01,744.950012,50.339031
4,2016-05-01,735.719971,48.306515
6,2016-07-01,768.789978,52.021957
7,2016-08-01,767.049988,52.737858


## 4. Agrupación con groupby()

Como no tenemos una columna 'Ticker', la agrupación más natural es por tiempo

### Agrupación por Componentes de la Fecha:

In [ ]:
# Necesitamos extraer componentes de la fecha (Año, Mes, Trimestre) para agrupar.
df_stocks['Year'] = df_stocks['formatted_date'].dt.year
df_stocks['Month'] = df_stocks['formatted_date'].dt.month
df_stocks['Quarter'] = df_stocks['formatted_date'].dt.quarter # Trimestre

In [ ]:
df_stocks.head()

,formatted_date,MCD,SBUX,GOOG,AMZN,MSFT,JPM,BAC,C,MAR,HLT,RCL,V,MA,PYPL,Year,Month,Quarter
0,2016-01-01,106.332146,54.353962,742.950012,587.000000,49.853489,50.424938,12.573010,36.897804,57.754189,35.192841,74.235298,71.574371,85.822624,36.139999,2016,1,1
1,2016-02-01,100.671043,52.064243,697.770020,552.520020,46.043667,48.033066,11.132540,33.707108,64.228912,41.061607,67.360649,69.556580,83.956566,38.139999,2016,2,1
2,2016-03-01,108.782211,53.571442,744.950012,593.640015,50.339031,50.524323,12.021718,36.223217,67.336624,44.499886,74.790009,73.631477,91.278160,38.599998,2016,3,1
3,2016-04-01,109.483307,50.457645,693.010010,659.590027,45.453705,53.919910,12.995729,40.153545,66.305466,43.716049,70.465584,74.363144,93.683258,39.180000,2016,4,2
4,2016-05-01,105.648926,49.255203,735.719971,722.789978,48.306515,56.098225,13.201019,40.447887,62.474155,41.198154,70.456474,75.999847,92.817329,37.790001,2016,5,2


In [ ]:
ticker_cols = ['MCD', 'SBUX', 'GOOG', 'AMZN', 'MSFT', 'JPM', 'BAC', 'C', 'MAR', 'HLT', 'RCL', 'V', 'MA', 'PYPL']

# Calcular el precio PROMEDIO anual para CADA ticker
# Agrupamos por año, y luego calculamos la media de todas las columnas numéricas (tickers)
promedio_anual_tickers = df_stocks.groupby('Year')[ticker_cols].mean()

promedio_anual_tickers.head()

,MCD,SBUX,GOOG,AMZN,MSFT,JPM,BAC,C,MAR,HLT,RCL,V,MA,PYPL
Year,,,,,,,,,,,,,,
2016,104.280773,51.142220,744.502492,707.225001,51.140350,57.396330,14.090748,40.754643,66.758651,44.944328,69.963773,74.984400,93.404650,38.510833
2017,134.991940,52.246241,928.541667,985.324992,68.682873,82.015368,22.678580,58.288383,101.931187,64.211220,104.028206,95.914673,125.499645,57.047500
2018,153.070151,53.496154,1113.346664,1657.460826,97.693677,99.390544,27.156693,61.179863,125.909911,77.314111,109.080777,130.577713,188.929565,83.079166
2019,187.446757,78.604043,1192.374176,1794.900838,128.960913,107.028756,28.159950,63.182767,129.675998,92.040335,114.404639,165.799440,257.973571,105.914166
2020,193.373987,80.814653,1487.327504,2728.572489,192.007441,100.823367,25.041814,49.861960,102.831286,87.902158,64.045230,191.706842,306.657092,166.795832


In [ ]:
# Calcular el precio MÁXIMO por trimestre para CADA ticker
maximo_trimestral_tickers = df_stocks.groupby(['Year', 'Quarter'])[ticker_cols].max()

maximo_trimestral_tickers.head()

MCD       SBUX        GOOG        AMZN       MSFT  \
Year Quarter                                                             
2016 1        108.782211  54.353962  744.950012  593.640015  50.339031   
     2        109.483307  51.435623  735.719971  722.789978  48.306515   
     3        102.580368  52.273071  777.289978  837.309998  53.195850   
     4        107.785545  52.382828  784.539978  789.820007  57.776409   
2017 1        115.617065  53.245609  829.559998  886.539978  61.606422   

                    JPM        BAC          C        MAR        HLT  \
Year Quarter                                                          
2016 1        50.524323  12.573010  36.897804  67.336624  44.499886   
     2        56.098225  13.201019  40.447887  66.305466  44.811901   
     3        58.466496  14.454847  41.614410  68.132416  47.477150   
     4        75.285149  19.958466  51.975777  79.212616  54.422619   
2017 1        79.500404  22.288458  52.466637  90.539024  57.002243   

                    RCL          V          MA       PYPL  
Year Quarter                                               
2016 1        74.790009  73.631477   91.278160  38.599998  
     2        70.465584  75.999847   93.683258  39.180000  
     3        68.566833  79.899200   98.711182  40.970001  
     4        75.560646  79.715660  103.803398  41.660000  
2017 1        90.880608  86.204704  109.521225  43.020000

In [ ]:
# Contar cuántos registros (meses) hay por año
conteo_anual = df_stocks.groupby('Year').size()
conteo_anual

,0
Year,
2016,12
2017,12
2018,12
2019,12
2020,12
2021,11


### Agregaciones Múltiples con .agg()

Aplicamos .agg() después de agrupar por tiempo, especificando qué hacer con cada columna de ticker (o las mismas operaciones para varias).

In [ ]:
# Calcular el precio promedio y la desviación estándar anual para GOOG y MSFT

agregaciones_seleccion = {
    'GOOG': ['mean', 'std'], # Media y desviación estándar para Google
    'MSFT': ['mean', 'std'],
    # Media y desviación estándar para Microsoft
}

promedio_desviacion_anual_tickers = df_stocks.groupby('Year').agg(agregaciones_seleccion)

promedio_desviacion_anual_tickers.head()

GOOG                    MSFT           
             mean         std        mean        std
Year                                                
2016   744.502492   33.530133   51.140350   3.996058
2017   928.541667   80.793320   68.682873   7.628497
2018  1113.346664   71.290477   97.693677   8.036377
2019  1192.374176   80.458189  128.960913  15.814371
2020  1487.327504  176.486158  192.007441  23.543889

In [ ]:
# Calcular el promedio anual para todos los tickers, pero el máximo anual solo para AMZN

# Definimos agregaciones para todas las columnas de tickers y una específica para AMZN
agg_dict = {ticker: 'mean' for ticker in ticker_cols} # Promedio para todos

agg_dict['AMZN'] = ['mean', 'max'] # Promedio y Max para AMZN (sobrescribe la anterior)

In [ ]:
agg_dict

{'MCD': 'mean',
 'SBUX': 'mean',
 'GOOG': 'mean',
 'AMZN': ['mean', 'max'],
 'MSFT': 'mean',
 'JPM': 'mean',
 'BAC': 'mean',
 'C': 'mean',
 'MAR': 'mean',
 'HLT': 'mean',
 'RCL': 'mean',
 'V': 'mean',
 'MA': 'mean',
 'PYPL': 'mean'}

In [ ]:
resumen_combinado = df_stocks.groupby('Year').agg(agg_dict)

In [ ]:
resumen_combinado

MCD        SBUX         GOOG         AMZN               \
            mean        mean         mean         mean          max   
Year                                                                  
2016  104.280773   51.142220   744.502492   707.225001   837.309998   
2017  134.991940   52.246241   928.541667   985.324992  1176.750000   
2018  153.070151   53.496154  1113.346664  1657.460826  2012.709961   
2019  187.446757   78.604043  1192.374176  1794.900838  1926.520020   
2020  193.373987   80.814653  1487.327504  2728.572489  3450.959961   
2021  228.915776  109.778783  2487.513639  3316.980003  3507.070068   

            MSFT         JPM        BAC          C         MAR         HLT  \
            mean        mean       mean       mean        mean        mean   
Year                                                                         
2016   51.140350   57.396330  14.090748  40.754643   66.758651   44.944328   
2017   68.682873   82.015368  22.678580  58.288383  101.931187   64.211220   
2018   97.693677   99.390544  27.156693  61.179863  125.909911   77.314111   
2019  128.960913  107.028756  28.159950  63.182767  129.675998   92.040335   
2020  192.007441  100.823367  25.041814  49.861960  102.831286   87.902158   
2021  271.868816  153.296657  39.697534  68.091782  143.445455  126.183636   

             RCL           V          MA        PYPL  
            mean        mean        mean        mean  
Year                                                  
2016   69.963773   74.984400   93.404650   38.510833  
2017  104.028206   95.914673  125.499645   57.047500  
2018  109.080777  130.577713  188.929565   83.079166  
2019  114.404639  165.799440  257.973571  105.914166  
2020   64.045230  191.706842  306.657092  166.795832  
2021   82.925454  218.815499  350.427732  253.879091

## 5 Combinando Filtrado y Agrupación

Filtra primero, luego agrupa; o agrupa primero, luego filtra los resultados.

In [ ]:
# Calcular el precio promedio trimestral de 'JPM' y 'BAC' solo para el año 2016

df_filtrado_2016 = df_stocks[df_stocks['Year'] == 2016]

promedio_trim_bancos_2016 = df_filtrado_2016.groupby('Quarter')[['JPM', 'BAC']].mean()

promedio_trim_bancos_2016.head()

,JPM,BAC
Quarter,,
1,49.660776,11.909089
2,54.475398,12.680358
3,57.041890,13.837736
4,68.407256,17.935809


## 6 Transformación a Formato Largo (melt)

In [ ]:
# Ejemplo rápido de cómo transformar a formato largo
df_largo = pd.melt(df_stocks,
                   id_vars=['formatted_date', 'Year', 'Month', 'Quarter'], # Columnas a mantener fijas
                   value_vars=ticker_cols,        # Columnas a 'despivotar'
                   var_name='Ticker',             # Nombre de la nueva columna para los nombres de las anteriores
                   value_name='Price')            # Nombre de la nueva columna para los valores


In [ ]:
print("\nDataFrame transformado a formato Largo (primeras 10 filas):")
df_largo.head(10)


DataFrame transformado a formato Largo (primeras 10 filas):


,formatted_date,Year,Month,Quarter,Ticker,Price
0,2016-01-01,2016,1,1,MCD,106.332146
1,2016-02-01,2016,2,1,MCD,100.671043
2,2016-03-01,2016,3,1,MCD,108.782211
3,2016-04-01,2016,4,2,MCD,109.483307
4,2016-05-01,2016,5,2,MCD,105.648926
5,2016-06-01,2016,6,2,MCD,104.160179
6,2016-07-01,2016,7,3,MCD,102.580368
7,2016-08-01,2016,8,3,MCD,100.845268
8,2016-09-01,2016,9,3,MCD,101.365387
9,2016-10-01,2016,10,4,MCD,98.913841


In [ ]:
precio_promedio_largo = df_largo.groupby('Ticker')['Price'].mean()

precio_promedio_largo

,Price
Ticker,
AMZN,1844.628025
BAC,25.946568
C,56.735507
GOOG,1309.236057
HLT,81.478392
JPM,99.241065
MA,218.651822
MAR,111.312456
MCD,166.141364
